In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import scipy.optimize as opt
from sklearn.linear_model import LogisticRegression

In [2]:
# DO NOT RE-RUN THIS LINE
# df = pd.read_csv('data.txt', sep="	")

In [37]:
# DO NOT RE-RUN THIS LINE
# all_unique_npi = df.npi.unique()

In [34]:
# Only grab providers who are individuals
# df = df[df.nppes_provider_first_name.notnull()]

In [40]:
all_specialties = df.specialty_description.unique()
index_for_specialty = dict((specialty,i) for i, specialty in enumerate(all_specialties))

all_generic_names = df.generic_name.unique()
X_header = np.insert(all_generic_names, 0, 'npi')
feature_size = X_header.shape[0]
generic_name_to_index = {}
for i, generic_name in enumerate(X_header):
    if i == 0:
        continue
    generic_name_to_index[generic_name] = i

In [41]:
# df.npi.unique().shape  ==== >> (807973,)
s0 = all_unique_npi[0:100]
#s1 = all_unique_npi[100001:200000]
#s2 = all_unique_npi[200001:300000]
#s3 = all_unique_npi[300001:400000]
#s4 = all_unique_npi[400001:500000]
#s5 = all_unique_npi[500001:600000]
#s6 = all_unique_npi[600001:700000]
#s7 = all_unique_npi[700001:900000]
# DOES NOT ERROR USING A INDEX > THAN 

In [42]:
X = pd.DataFrame(columns=X_header)
y = np.zeros(s0.shape[0])

In [43]:
for i, npi in enumerate(s0):
    npi_matches = df.loc[df['npi'] == npi]

    speciality = npi_matches.specialty_description.values[0]
    y[i] = index_for_specialty[speciality]

    # We need to normalize bene_count across doctor's panel otherwise might be getting strange numbers
    total_bene_counts = npi_matches.bene_count.fillna(5).sum()
    feature_vector = np.zeros(feature_size)
    feature_vector[0] = npi
    for generic_name in npi_matches.generic_name.unique():
        generic_name_matches = npi_matches.loc[npi_matches['generic_name'] == generic_name]
        index_for_feature_vector = generic_name_to_index[generic_name]
        normalized_bene_count = generic_name_matches.bene_count.fillna(5).sum() / total_bene_counts
        feature_vector[index_for_feature_vector] = normalized_bene_count
    X.loc[i] = feature_vector

In [44]:
# https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file
#X.to_csv("features_for_s0.txt", sep='\t')
#pd.DataFrame(y).to_csv("labels_for_s0.txt", sep='\t')
#meow0 = pd.read_csv('features_for_s0.txt', sep="\t")
#meow1 = pd.read_csv('labels_for_s0.txt', sep="\t")
#meow0
#meow1
#meow0.append(meow1)

In [45]:
X_without_npi = X.drop(columns=['npi'])

In [46]:
#clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = clf.fit(X_without_npi, y_transpose)

ValueError: Found input variables with inconsistent numbers of samples: [100, 9]

In [ ]:
X_without_npi